In [57]:
from google.colab import drive
drive.mount('/content/drive')
import numpy as np
import pandas as pd
import math
import warnings
warnings.filterwarnings("ignore")
# I have used my google drive to load the training and testing data
# Upload the dataset file in your google drive and change the path to run the below line
data= pd.read_csv('/content/drive/MyDrive/ML/Priyadataset2c.csv',dtype='object')
print(data)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
                 height            diameter             weight  \
0      0.12493777936236    0.13190375787948   0.38616824929558   
1      0.14492137354649    0.12500746617994               0.75   
2     0.071892497830642                0.03    0.1631761874001   
3     0.084260612354435   0.037681879898677   0.23151165834924   
4                  0.07   0.090347950123449   0.19177699658224   
..                  ...                 ...                ...   
115    0.15324808700145     0.1301638325634   0.40118457408591   
116    0.12021522833926    0.11607358210855   0.33882777489585   
117    0.07556917126148   0.072067954099429   0.16160560001853   
118    0.13137129130651   0.091788907990009    0.2781412506351   
119    0.12825404991764    0.14820139358115   0.45533536592389   

                   hue      class  
0      2.9517669794008   Plastic   
1     

Using first 100 records as training set and remaining 20 records as test data set

In [58]:
X_train=data.iloc[0:100,:-1]
print(X_train)
X_test = data.iloc[100:,:-1]
print(X_test)
X_test=X_test.astype('float64')
X_train=X_train.astype('float64')
X = data.iloc[:,:-1]
Y = data.iloc[:,-1]
x_train = X.iloc[0:100,:].values
x_test = X.iloc[100:,:].values
y_train = Y[0:100].values
y_test = Y[100:].values
x_test=x_test.astype('float64')
x_train=x_train.astype('float64')

                height            diameter             weight  \
0     0.12493777936236    0.13190375787948   0.38616824929558   
1     0.14492137354649    0.12500746617994               0.75   
2    0.071892497830642                0.03    0.1631761874001   
3    0.084260612354435   0.037681879898677   0.23151165834924   
4                 0.07   0.090347950123449   0.19177699658224   
..                 ...                 ...                ...   
95                0.05   0.063594729215563   0.13143726276907   
96    0.14203794136301    0.11033482248778               0.75   
97   0.059841975005583     0.1196374053952   0.43597686894734   
98    0.11622427723581   0.089264682323942   0.64108492044275   
99    0.10430398237586   0.097649309187052   0.40579556041064   

                 hue  
0    2.9517669794008  
1    3.4378755151121  
2    4.0521528606463  
3    6.2831853071796  
4    2.1255038511543  
..               ...  
95   3.6012437646385  
96   2.9244966386833  
97   2.39016

In [59]:
for i in range(len(y_train)):
    if y_train[i] == ' Plastic ':
        y_train[i] = 0
    elif y_train[i] == ' Metal ':
        y_train[i] = 1
    elif y_train[i] == ' Ceramic ':
        y_train[i] = 2

In [60]:
for i in range(len(y_test)):
    if y_test[i] == ' Plastic ':
        y_test[i] = 0
    elif y_test[i] == ' Metal ':
        y_test[i] = 1
    elif y_test[i] == ' Ceramic ':
        y_test[i] = 2

Softmax Regression classifier from project 1 which I implemented

In [61]:
# doing one hot coding
# representing the y values in the form of a matrix and c represent the number classes
# the matrix is of size  len(y) and no. of classes
def one_hot(y, c):
    y_hot=(np.arange(np.max(y) + 1) == y[:, None]).astype(float)
    return y_hot

In [62]:
# implementing the softmax regression probability function
def softmax(a):
    Pro_exp = np.exp(a - np.max(a))
    for i in range(len(a)):
        Pro_exp[i] /= np.sum(Pro_exp[i])
    return Pro_exp

In [63]:
# using xtraining data to train the algorithm
# fitting the function by using random values for weight and passing 1 as bias and updating the graident
def fit(X, y, lr, c, epochs):
    m, n = X.shape
    # Initializing weights randomly.
    w = np.random.uniform(low=-0.1, high=0.1, size=(n,c))
    b = np.array([1, 1, 1])
    for epoch in range(epochs):
        y_hot = one_hot(y, c)
        z_val = np.dot(X,w)+ b
        y_pred = softmax(z_val)
        w_gradient = (1/m)*np.dot(X.T, (y_pred - y_hot)) 
        w = w - lr*w_gradient
    return w

In [64]:
# prediction function
def predict(X, w):
    b = np.array([1, 1, 1])
    z_val = X@w + b
    y_predictions = softmax(z_val)
    return np.argmax(y_predictions, axis=1)

In [65]:
# calculate the accuracy
def Accuracy(y, y_prds):
    return (np.sum(y==y_prds)/len(y))*100

Without using Bagging -( single classifier)

In [66]:
clsr = fit(x_train, y_train, lr=0.8, c=3, epochs=6000)
#print(clsr)

In [67]:
test_pred=(predict(x_test, clsr))
print(test_pred)

[0 0 0 0 2 2 2 0 2 2 2 2 2 0 0 0 0 0 0 0]


In [68]:
print(f'Accuracy : {Accuracy(y_test, test_pred)}')

Accuracy : 95.0


1a) Implementing a bagging routine for a softmax regression classifier. 
(softmax regression classifier that i used in project 1)

Using Bagging on softmax regression

In [69]:
# bagging function
import collections
def bagging(n_bg):
    data = []
    for i in range(n_bg):
      # randomly selecting the datapoints from the training dataset
        idx = np.random.randint(0,100,100)
        dta= X_train.iloc[idx]
        data.append(dta)
    clr_list = []
    for d in data:
        x_train = d.iloc[:,0:4].values
        clsr = fit(x_train, y_train, lr=0.8, c=3, epochs=6000)
        # append the classifier
        clr_list.append(clsr)
    c = 0
    predict1 =[]
    for lc in clr_list:
      e = predict(x_test, lc)
      predict1.append(e)
    #print(predict1)
    result=[]
    for j in range(len(y_test)):
      pred_i=[p[j] for p in predict1]
      f_pred= collections.Counter(pred_i).most_common(1)[0][0]
      result.append(f_pred)
    for k in range(len(y_test)):
      if result[k] == y_test[k]:
            c += 1
    Acc = (c / len(y_test)) * 100
    return Acc

1b) Applying bagging 10, 50, and 100 times to the training data.

In [70]:
n = [10, 50, 100]
for n_bg in n:
    Accuracy = bagging(n_bg)
    print(f'Bagging value of : {n_bg} for softmax regression has Accuracy of : {Accuracy} ')

Bagging value of : 10 for softmax regression has Accuracy of : 50.0 
Bagging value of : 50 for softmax regression has Accuracy of : 55.00000000000001 
Bagging value of : 100 for softmax regression has Accuracy of : 55.00000000000001 


Before implementing bagging (i.e on single classifier), the accuracy was 95%.
After implementing bagging, the accuracy is reduced as seen in above result.
Implementation of bagging has decreased the accuracy rate for softmax regression. hence Bagging degraded the performance.